# Taku Ito
## 12/12/2018

## Compute pairwise PCA FC (500 components)


In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import h5py
os.environ['OMP_NUM_THREADS'] = str(1)
import multiprocessing as mp
import scipy.stats as stats
import time
os.sys.path.append('utils/')
from sklearn.decomposition import PCA
import statsmodels.api as sm

sns.set_style("white")
plt.rcParams["font.family"] = "FreeSans"

/home/tito/miniconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# Load resting-state data

In [3]:
def loadRestActivity(subj,model='24pXaCompCorXVolterra',zscore=False):
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['Rest1/nuisanceReg_resid_24pXaCompCorXVolterra'][:].copy()
    h5f.close()
    
    if zscore:
        data = stats.zscore(data,axis=1)
    return data

def loadMask(roi,dilated=True):
    maskdir = basedir + 'data/results/surfaceMasks/'
    if dilated:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '_dilated_10mm.dscalar.nii'
    else:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '.dscalar.nii'
    maskdata = np.squeeze(nib.load(maskfile).get_data())
    return maskdata


def pcaFC(stim,resp,n_components=500,nproc=10):
#     print '\tRunning PCA'
    os.environ['OMP_NUM_THREADS'] = str(nproc)
    pca = PCA(n_components)
    reduced_mat = pca.fit_transform(stim) # Time X Features
    
    inputs = []
    for vert in range(resp.shape[1]):
        inputs.append((resp[:,vert],reduced_mat,True))

#     print '\tRunning regression'
    os.environ['OMP_NUM_THREADS'] = str(1)
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(_regression2,inputs).get()
    pool.close()
    pool.join()
    
    
    wt = np.zeros((stim.shape[1],resp.shape[1]))
    vert = 0
    for result in results:
        betas, resid = result
        betas = pca.inverse_transform(betas[1:])
        wt[:,vert] = betas
        vert += 1

    return wt

def multRegFC(stim,resp,nproc=10):
    
    inputs = []
    for vert in range(resp.shape[1]):
        inputs.append((resp[:,vert],stim,True))

#     print '\tRunning regression'
    os.environ['OMP_NUM_THREADS'] = str(1)
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(_regression2,inputs).get()
    pool.close()
    pool.join()
    
    
    wt = np.zeros((stim.shape[1],resp.shape[1]))
    vert = 0
    for result in results:
        betas, resid = result
        betas = betas[1:]
        wt[:,vert] = betas
        vert += 1

    return wt


def _regression2((data,regressors,constant)):
    """ 
    Hand coded OLS regression using closed form equation: betas = (X'X)^(-1) X'y
    """
    # Add 'constant' regressor
    if constant:
        regressors = sm.add_constant(regressors)
    X = regressors.copy()
    try:
#        #C_ss_inv = np.linalg.inv(np.dot(X.T,X))
        C_ss_inv = np.linalg.pinv(np.dot(X.T,X))
    except np.linalg.LinAlgError as err:
        C_ss_inv = np.linalg.pinv(np.cov(X.T))
    betas = np.dot(C_ss_inv,np.dot(X.T,data.T))
    resid = data - (betas[0] + np.dot(X[:,1:],betas[1:])).T
    return betas, resid

# Now for each subject fit the regression model using the optimized penalty term (alpha), and save to disk

In [4]:
# ROI to compute FC to
target_rois = [9,189,8,188] 
# Number of parallel processes
nproc = 10
# Number of PCs to use
nPCs = 500

pcafcdir = '/projects3/SRActFlow/data/results/pcaFC/pairwiseFC/'

for target_roi in target_rois:
    
    print 'Running pairwise PC regression for target region', target_roi
    # Now load regular mask to identify responses/activities we want to predict (target data)
    mask = loadMask(target_roi,dilated=False)
    target_ind = np.where(mask)[0]

    scount = 1
    for subj in subjNums:
        if scount%10==0:
            print 'Subject', scount, '/', len(subjNums), 'time elapsed...', (timeend-timestart), 'seconds'
        subjData = loadRestActivity(subj,zscore=False)

        targetMask = loadMask(target_roi,dilated=True)
        targetMask_ind = np.where(targetMask==1)[0]
        
        timestart = time.time()
        for source_roi in range(1,nParcels+1):
            if source_roi==target_roi: continue
#             print 'Running source ROI', source_roi

            source_ind = np.where(glasser2==source_roi)[0]
            # Make sure it doesn't overlap with the dilated mask
    #         source_ind = np.intersect1d(np.where(targetMask==0)[0],source_ind)
            adjacent_vertices = np.intersect1d(targetMask_ind,source_ind)

            tmp_data = subjData.copy()
            # Make sure to 0 out any adjacent vertices
            if len(adjacent_vertices)>0:
                tmp_data[adjacent_vertices,:] = 0
            # Gather source and target data for pc regression
            sourceData = tmp_data[source_ind,:].copy()
            targetData = tmp_data[target_ind,:].copy()

#             print '\tRunning source to target PCA regression using', nproc, 'processes'
            if len(source_ind)<nPCs:
                sourceToTargetMappings = multRegFC(sourceData.T,targetData.T,nproc=nproc)
            else:
                sourceToTargetMappings = pcaFC(sourceData.T,targetData.T,n_components=nPCs,nproc=nproc)

            # All vertices to target indices (makes it easier for comparison)
            sourceToTargetMappings2 = sourceToTargetMappings

            # Save out to file
#             print '\tSaving out to disk'
            h5f = h5py.File(pcafcdir + 'SourceParcel' + str(source_roi) + 'TargetParcel' + str(target_roi) + '_pcaPairwiseFC_nozscore.h5','a')
            try:
                h5f.create_dataset(subj + '/sourceToTargetMapping',data=sourceToTargetMappings2)
            except:
                del h5f[subj+'/sourceToTargetMapping']
                h5f.create_dataset(subj + '/sourceToTargetMapping',data=sourceToTargetMappings2)
            h5f.close()

            del sourceToTargetMappings2, sourceToTargetMappings
            
        scount += 1
        timeend = time.time()


Running pairwise PC regression for target region 9
Subject 10 / 96 time elapsed... 427.708504915 seconds
Subject 20 / 96 time elapsed... 440.251296043 seconds
Subject 30 / 96 time elapsed... 471.877815962 seconds
Subject 40 / 96 time elapsed... 444.693813801 seconds
Subject 50 / 96 time elapsed... 529.85731411 seconds
Subject 60 / 96 time elapsed... 770.920521975 seconds
Subject 70 / 96 time elapsed... 749.860127926 seconds
Subject 80 / 96 time elapsed... 716.69156599 seconds
Subject 90 / 96 time elapsed... 790.216306925 seconds
Running pairwise PC regression for target region 189
Subject 10 / 96 time elapsed... 723.680526972 seconds
Subject 20 / 96 time elapsed... 589.446938038 seconds
Subject 30 / 96 time elapsed... 476.93251586 seconds
Subject 40 / 96 time elapsed... 752.841898918 seconds
Subject 50 / 96 time elapsed... 471.984141827 seconds
Subject 60 / 96 time elapsed... 406.215167999 seconds
Subject 70 / 96 time elapsed... 405.478977919 seconds
Subject 80 / 96 time elapsed... 407